In [19]:
# -*- coding: utf-8 -*-
"""
Created on Thu Jul 23 2015

@author: Anton at 0x0af@ukr.net
"""

import numpy
import scipy
import copy, math
from scipy.fftpack import dct, idct
from scipy.linalg import schur
from PIL import Image

In [20]:
def SchurDecomposition(GrayscaleImage):
    """
     Schur decomposition. Input a grayscale image to get it's upper triangular matrix and unitary matrix.
     
     Schur's theorem announced that : if A ∈ Cn×n, there exists a unitary matrix U and an upper triangular matrix T,
     such that: A = U × T × U'
     
     0x0af@ukr.net, 23-July-2015
    """
    
    # TODO: implement Schur decomposition
    
    TriangularMatrix = GrayscaleImage
    UnitaryMatrix = GrayscaleImage
    
    return TriangularMatrix, UnitaryMatrix

In [21]:
def InverseSchurDecomposition(UnitaryMatrix,TriangularMatrix):
    """
     Inverse Schur decomposition. Input a 2D unitary matrix and a 2D triangular matrix to get the original grayscale image
    
     23-July-2015
    """
    
    # TODO: implement inverse Schur decomposition
    
    return OriginalGrayscaleImage

In [36]:
def RazafindradinaEmbed(GrayscaleContainerPath,GrayscaleWatermarkPath,Alpha):
    """    
    Razafindradina embedding method implementation. 
    
    Outputs the resulting watermarked image
    
    23-July-2015
    """
    
    grayscaleContainer2DArray = numpy.asarray(Image.open(GrayscaleContainerPath).convert("L"))
    grayscaleWatermark2DArray = numpy.asarray(Image.open(GrayscaleWatermarkPath).convert("L"))
    
    assert (grayscaleContainer2DArray.shape[0]==grayscaleContainer2DArray.shape[1]) and (grayscaleContainer2DArray.shape[0]==grayscaleWatermark2DArray.shape[0]) and (grayscaleContainer2DArray.shape[1]==grayscaleWatermark2DArray.shape[1]), 'GrayscaleContainer and GrayscaleWatermark sizes do not match or not square' 
    
    # Perform DCT on GrayscaleContainer
    
    #print grayscaleContainer2DArray
    
    grayscaleContainer2DArray = dct(dct(grayscaleContainer2DArray,axis=0),axis=1)
    
    #print grayscaleContainer2DArray
    
    # Perform SchurDecomposition on GrayscaleWatermark
    
    # Alpha-blend GrayscaleWatermark TriangularMatrix into GrayscaleContainer DCT coeffs with Alpha
    
    # Perform IDCT on GrayscaleContainer DCT coeffs to get WatermarkedImage
    
    watermarkedImage = Image.fromarray(numpy.uint8(grayscaleContainer2DArray))
    #watermarkedImage.show()
    
    # Write image to file
    
    watermarkedImage.save(WatermarkedImagePath)
    
    return UnitaryMatrix

In [37]:
def RazafindradinaExtract(GrayscaleStegoPath, GrayscaleContainerPath, ExtractedGrayscaleWatermarkPath, UnitaryMatrix, Alpha):
    """
    Rezefindradina extracting method implementation.
    
    Outputs the extracted watermark
    
    23-July-2015
    """
    
    grayscaleContainer2DArray = numpy.asarray(Image.open(GrayscaleContainerPath).convert("L"))
    grayscaleStego2DArray = numpy.asarray(Image.open(GrayscaleStegoPath).convert("L"))
    
    # Perform DCT on GrayscaleContainer
    
    # Perform DCT on GrayscaleStego
    
    # Extract GrayscaleWatermark TriangularMatrix with Alpha
    
    # Perform InverseSchurDecomposition on UnitaryMatrix and TrinagularMatrix to get ExtractedGrayscaleWatermark
    
    extractedGrayscaleWatermark = Image.fromarray(numpy.uint8(GrayscaleWatermark))
    #watermarkedImage.show()
    
    # Write image to file
    
    extractedGrayscaleWatermark.save(ExtractedGrayscaleWatermarkPath)
    
    return

In [38]:
RazafindradinaEmbed("C:\Users\Tony\Documents\GitHub\steganography-embedding\GrayscaleContainer.bmp","C:\Users\Tony\Documents\GitHub\steganography-embedding\SameSizeGrayscaleWatermark.bmp",0.1)

ValueError: dtype uint8 not supported